# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 5 2022 2:10PM,247980,10,8614406,Yusen – 3D Matrix,Released
1,Oct 5 2022 2:10PM,247979,10,8614405,"ClearSpec, LLC",Released
2,Oct 5 2022 2:08PM,247978,10,8614402,"ClearSpec, LLC",Released
3,Oct 5 2022 2:07PM,247977,10,8614404,Yusen – 3D Matrix,Released
4,Oct 5 2022 2:04PM,247975,10,0086057263,ISDIN Corporation,Released
5,Oct 5 2022 2:04PM,247975,10,0086057264,ISDIN Corporation,Released
6,Oct 5 2022 2:04PM,247975,10,0086057286,ISDIN Corporation,Released
7,Oct 5 2022 2:04PM,247975,10,0086057289,ISDIN Corporation,Released
8,Oct 5 2022 2:04PM,247975,10,0086057287,ISDIN Corporation,Released
9,Oct 5 2022 2:04PM,247975,10,0086057288,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,247976,Released,1
38,247977,Released,1
39,247978,Released,1
40,247979,Released,1
41,247980,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247976,NaN,NaN,1.0
247977,NaN,NaN,1.0
247978,NaN,NaN,1.0
247979,NaN,NaN,1.0
247980,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247841,28.0,30.0,3.0
247886,0.0,6.0,0.0
247888,0.0,0.0,6.0
247889,0.0,0.0,5.0
247892,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247841,28,30,3
247886,0,6,0
247888,0,0,6
247889,0,0,5
247892,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247841,28,30,3
1,247886,0,6,0
2,247888,0,0,6
3,247889,0,0,5
4,247892,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247841,28,30,3
1,247886,,6,
2,247888,,,6
3,247889,,,5
4,247892,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix
1,Oct 5 2022 2:10PM,247979,10,"ClearSpec, LLC"
2,Oct 5 2022 2:08PM,247978,10,"ClearSpec, LLC"
3,Oct 5 2022 2:07PM,247977,10,Yusen – 3D Matrix
4,Oct 5 2022 2:04PM,247975,10,ISDIN Corporation
30,Oct 5 2022 2:03PM,247976,16,"SHL Pharma, LLC"
31,Oct 5 2022 1:54PM,247974,19,ACG North America LLC
34,Oct 5 2022 1:51PM,247973,10,"Methapharm, Inc."
42,Oct 5 2022 1:51PM,247952,10,"Methapharm, Inc."
52,Oct 5 2022 1:45PM,247972,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix,,,1
1,Oct 5 2022 2:10PM,247979,10,"ClearSpec, LLC",,,1
2,Oct 5 2022 2:08PM,247978,10,"ClearSpec, LLC",,,1
3,Oct 5 2022 2:07PM,247977,10,Yusen – 3D Matrix,,,1
4,Oct 5 2022 2:04PM,247975,10,ISDIN Corporation,,,26
5,Oct 5 2022 2:03PM,247976,16,"SHL Pharma, LLC",,,1
6,Oct 5 2022 1:54PM,247974,19,ACG North America LLC,,,3
7,Oct 5 2022 1:51PM,247973,10,"Methapharm, Inc.",,,8
8,Oct 5 2022 1:51PM,247952,10,"Methapharm, Inc.",,,10
9,Oct 5 2022 1:45PM,247972,10,ISDIN Corporation,,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix,1,,
1,Oct 5 2022 2:10PM,247979,10,"ClearSpec, LLC",1,,
2,Oct 5 2022 2:08PM,247978,10,"ClearSpec, LLC",1,,
3,Oct 5 2022 2:07PM,247977,10,Yusen – 3D Matrix,1,,
4,Oct 5 2022 2:04PM,247975,10,ISDIN Corporation,26,,
5,Oct 5 2022 2:03PM,247976,16,"SHL Pharma, LLC",1,,
6,Oct 5 2022 1:54PM,247974,19,ACG North America LLC,3,,
7,Oct 5 2022 1:51PM,247973,10,"Methapharm, Inc.",8,,
8,Oct 5 2022 1:51PM,247952,10,"Methapharm, Inc.",10,,
9,Oct 5 2022 1:45PM,247972,10,ISDIN Corporation,14,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix,1,,
1,Oct 5 2022 2:10PM,247979,10,"ClearSpec, LLC",1,,
2,Oct 5 2022 2:08PM,247978,10,"ClearSpec, LLC",1,,
3,Oct 5 2022 2:07PM,247977,10,Yusen – 3D Matrix,1,,
4,Oct 5 2022 2:04PM,247975,10,ISDIN Corporation,26,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix,1.0,NaN,NaN
1,Oct 5 2022 2:10PM,247979,10,"ClearSpec, LLC",1.0,NaN,NaN
2,Oct 5 2022 2:08PM,247978,10,"ClearSpec, LLC",1.0,NaN,NaN
3,Oct 5 2022 2:07PM,247977,10,Yusen – 3D Matrix,1.0,NaN,NaN
4,Oct 5 2022 2:04PM,247975,10,ISDIN Corporation,26.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix,1.0,0.0,0.0
1,Oct 5 2022 2:10PM,247979,10,"ClearSpec, LLC",1.0,0.0,0.0
2,Oct 5 2022 2:08PM,247978,10,"ClearSpec, LLC",1.0,0.0,0.0
3,Oct 5 2022 2:07PM,247977,10,Yusen – 3D Matrix,1.0,0.0,0.0
4,Oct 5 2022 2:04PM,247975,10,ISDIN Corporation,26.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4958965,169.0,19.0,0.0
15,495767,5.0,33.0,38.0
16,991639,12.0,6.0,0.0
19,1239846,23.0,21.0,0.0
22,495937,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4958965,169.0,19.0,0.0
1,15,495767,5.0,33.0,38.0
2,16,991639,12.0,6.0,0.0
3,19,1239846,23.0,21.0,0.0
4,22,495937,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,169.0,19.0,0.0
1,15,5.0,33.0,38.0
2,16,12.0,6.0,0.0
3,19,23.0,21.0,0.0
4,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,169.0
1,15,Released,5.0
2,16,Released,12.0
3,19,Released,23.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,22
Status,,,,,
Completed,0.0,38.0,0.0,0.0,0.0
Executing,19.0,33.0,6.0,21.0,0.0
Released,169.0,5.0,12.0,23.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,22
0,Completed,0.0,38.0,0.0,0.0,0.0
1,Executing,19.0,33.0,6.0,21.0,0.0
2,Released,169.0,5.0,12.0,23.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,22
0,Completed,0.0,38.0,0.0,0.0,0.0
1,Executing,19.0,33.0,6.0,21.0,0.0
2,Released,169.0,5.0,12.0,23.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()